In [1]:
import csv
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import re  # regular expressions
import nltk  # draw on the Python natural language toolkit
from ast import literal_eval
import time
from collections import Counter
from textblob import TextBlob

In [2]:
matplotlib.style.use('ggplot')
#%matplotlib inline 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.max_colwidth = 200
pd.set_option('display.width', 1000)

In [3]:
D_DIR = "../data/"
mcdonalds_icmr = "mcdonalds/icmr_mcdonalds_v2.csv"
mcdonalds_icmr_clean = "mcdonalds/df_mc_20161011.pkl"
burgerking_icmr_clean = "burgerking/df_bk_20161011.pkl"
mcdonalds_icmr_labels = "mcdonalds/icmr_mcdonalds_labels_v2.csv"
burgerking_icmr = "burgerking/icmr_burgerking_v2.csv"
burgerking_icmr_labels = "burgerking/icmr_burgerking_labels_v2.csv"
allFiles = [D_DIR + mcdonalds_icmr,D_DIR + burgerking_icmr]

In [4]:
df_mc_ = pd.read_pickle(D_DIR + mcdonalds_icmr_clean) # dump csv into a Pandas DataFrame
df_bk_ = pd.read_pickle(D_DIR + burgerking_icmr_clean) # dump csv into a Pandas DataFrame

In [5]:
df_bk_.head(10)

,instaGramId,imageDateCreated,imageTagCount,imageFilter,imageLikeCount,imageCommentCount,imageLink,imageCaption,imageUsersInImageCollection,imageTagCollection,imageLongitude,imageLatitude,userFullName,userProfileImage,userName,userId,followedByCount,followersCount,postedMedia,userBio,userPostedLikedImage,comments,googleLogo,googleFaceCount,googleText,googleLandMark,product,brand,googleLabel_list,googleLabelConfidence_list,googleLabel_dict,imageTagCollection_list,imageCaption_clean,famous_attribute,activity_attribute,sentiment_attribute_num,sentiment_attribute,imageDateCreated_str,weekend_attribute,filter_attribute,face_attribute,logo_attribute,food_attribute,conjoint_category
0,1165524631240624607_638926073,1453161384,7,Normal,3,0,https://scontent.cdninstagram.com/hphotos-xpt1/t51.2885-15/s320x320/e35/12545400_131527523893372_2086774170_n.jpg,I will gladly pay you Tuesday for a hamburger today. #popeye #popeyethesailorman #wimpy #wimpysburgers #jackinthebox #mcdonalds #burgerking,[],"[""burgerking"",""popeyethesailorman"",""mcdonalds"",""wimpysburgers"",""jackinthebox"",""popeye"",""wimpy""]",-89.823100,37.913900,"""Lisa Qualls Rebman""",https://scontent.cdninstagram.com/hphotos-xaf1/t51.2885-19/s150x150/12276988_191716141166756_530478826_a.jpg,"""lisarebs""",6.389261e+08,132,268,1111,"Stay at home mom to 3 kids. Active in pregnancy & infant loss support. Dealing with a spinal disease, trying to live a happy life with minimal pain.",False,[],[],0,"""IT'S TUESDAY BITCH!!!\nWHERE'S OUR-\nFUCKING MONEY!!!""",NaN,0,BurgerKing,"[cartoon, comics, artwork]","[0.92367387, 0.67196417, 0.70198238]","{u'comics': 0.67196417, u'cartoon': 0.92367387, u'artwork': 0.70198238}","[burgerking, popeyethesailorman, mcdonalds, wimpysburgers, jackinthebox, popeye, wimpy]","[i, gladly, pay, tuesday, hamburger, today, ]",common,low,0.500000,positive,2016-01-18 23:56:24,weekend no,filter no,face no,logo no,food no,0
1,1164403947432345237_188454382,1453027789,24,Normal,27,2,https://scontent.cdninstagram.com/hphotos-xpf1/t51.2885-15/s480x480/e35/11931269_474852249364531_1430106759_n.jpg,"regram @animalrevenge\nMcDonalds, Arby's, Wendy's, Taco Bell, Burger King and all these nasty fast food chains that only serve crap to our children can go to hell!\n\nWe need more fresh markets! ...",[],"[""wendys"",""bk"",""m"",""pic"",""archdaily"",""mcdonalds"",""greenhouse"",""vegans"",""mcshit"",""govegan"",""gogreen"",""burgerking"",""tacobell"",""arbys"",""picoftheday"",""meatlessmonday"",""freshmarket"",""nongmo"",""meatismur...",-73.591152,40.709365,Angele Lbn,https://scontent.cdninstagram.com/hphotos-xpa1/t51.2885-19/s150x150/12277478_956146861113938_31544961_a.jpg,ange_0,1.884544e+08,158,136,333,Angèle ; 19yo ; 🇫🇷\nIn love with animals 🐶🐱🐹🐰🐻🐼🐨🐔🐧🐴🐝🐛🐌🐞🐍🐢 i'm also a vegetarian 🌻🌍🌏🌎,False,"[""""Yes!"""",""""Ahhh!""""]",[],0,"""icDonald\nWE NEED LESS OF THESE\nAND MORE OF THESE\nMIND""",NaN,0,BurgerKing,"[amusement park, toy, fair, park]","[0.78556997, 0.62020761, 0.57394826, 0.52019644]","{u'toy': 0.62020761, u'park': 0.52019644, u'fair': 0.57394826, u'amusement park': 0.78556997}","[wendys, bk, m, pic, archdaily, mcdonalds, greenhouse, vegans, mcshit, govegan, gogreen, burgerking, tacobell, arbys, picoftheday, meatlessmonday, freshmarket, nongmo, meatismurder, vegansofig, mc...","[regram, mcdonalds, arby, wendy, taco, bell, burger, king, nasty, fast, food, chains, serve, crap, children, go, hell, need, fresh, markets, organic, gardens, greenhouses, follow, imagine, could, ...",common,high,-0.220000,negative,2016-01-17 10:49:49,weekend yes,filter no,face no,logo no,food no,0
2,1164092983056274899_1252201540,1452990719,20,Normal,65,2,https://scontent.cdninstagram.com/hphotos-xaf1/t51.2885-15/s480x480/e35/12479444_1127394633951162_1238237322_n.jpg,What my Saturday night SHOULD be all about!! Chicken Nuggets & Fries !! #PaulEats #PaulEatsNugs #NugsNotDrugs #Weekend,[],"[""foodie"",""food"",""wendys"",""goodeats"",""pauleats"",""weekend"",""mcdonalds"",""foodstagram""

In [14]:
df_mc_['conjoint_category'].value_counts()


0     27125
1       496
34      256
22      153
9        53
25       22
16       13
28        5
6         5
27        2
Name: conjoint_category, dtype: int64

In [13]:
df_bk_['conjoint_category'].value_counts()

0     27511
1       496
33       53
25       22
23       16
16       13
13        9
36        5
4         5
Name: conjoint_category, dtype: int64

Of all 36 combinantions only the following are used for MC:
1;34;22;9;25;16;28;6;27
Of all 36 combinantions only the following are used for BK:
1;33;25;23;16;13;36;4
join of groups:
1;4;6;9;13;16;22;23;25;27;28;33;34;36